# Подготовка данных до blastn

In [1]:
import pandas as pd
import pickle
from Bio import SeqIO
from collections import Counter
from sklearn.metrics import f1_score, classification_report
import numpy as np

In [39]:
PATH_SAMPLE_IDS = '/home/parazit/ml_virus_host/v2.0/sample_ids/' # путь к файлам, содержащим AC геномов тренировочной, валидайионной и тестовой выборок
PATH_SAMPLE_IDS_fr = PATH_SAMPLE_IDS+"ranged_genus/undersampling_7-20_fragments/" # путь к файлам, содержащим ID фрагментов тренировочной, валидайионной и тестовой выборок
PATH_DATA = '/home/parazit/ml_virus_host/v2.0/data/' # путь к файлам, содержащим геномы или их фрагменты в формате fasta
PATH_GENOMES2FRAGMENTS = "/home/parazit/ml_virus_host/v2.0/baseline/blast/ranged_genus/genomes2fragments/"

In [98]:
meta_df=pd.read_csv(PATH_DATA+'data_table.tsv', sep='\t', index_col = 'refseq id')
meta_df_ranged = pd.read_csv(PATH_DATA+'data_table_400.tsv', sep=',', index_col = 0)

# Подготовка полногеномных выборок для поиска фрагментов

Тестовая выборка состоит из фрагментов, геномых которых не используются при обучении

In [62]:
# Загружаем необходимые файлы и таблицы

fr800_train = pickle.load(open(PATH_SAMPLE_IDS_fr + "train_val_test_800_it0.pkl", "rb"))[0]
fr400_train = pickle.load(open(PATH_SAMPLE_IDS_fr + "train_val_test_400_it0.pkl", "rb"))[0]
fr800_test = pickle.load(open(PATH_SAMPLE_IDS_fr + "train_val_test_800_it0.pkl", "rb"))[2]
fr400_test = pickle.load(open(PATH_SAMPLE_IDS_fr + "train_val_test_400_it0.pkl", "rb"))[2]
meta_df = pd.read_csv(PATH_DATA+'data_table.tsv', sep='\t', index_col = 0)
meta_df_800 = pd.read_csv(PATH_DATA+'data_table_800.tsv', sep=',', index_col = 0)
meta_df_400 = pd.read_csv(PATH_DATA+'data_table_400.tsv', sep=',', index_col = 0)

In [6]:
def make_fasta_file(fragment_len, train_ids):
    with open(PATH_DATA+"data.fasta", "r") as data_file, \
        open("ranged_genus/genomes2fragments/train_sequences_"+fragment_len+".fasta", "w") as train_file:
        
        for seq in SeqIO.parse(data_file, "fasta"):
            if seq.id in train_ids:
                SeqIO.write(seq, train_file, "fasta")

def obtain_genomes_ids(fragments_train, meta_df):
    tmp, new_ids = [x[:9] for x in fragments_train], []
    
    for x in tmp:
        if x[-1] == "_":
            new_ids.append(x[:-1])
        else:
            new_ids.append(x)
    return new_ids

Выделяем AC геномов, фрагменты которых не использовались при обучении

In [7]:
genomes_train_800 = obtain_genomes_ids(fr800_train.index, meta_df) 
genomes_train_400 = obtain_genomes_ids(fr400_train.index, meta_df)

print(len(genomes_train_800), "800")
print(len(genomes_train_400), "400")

1162 800
1144 400


Сохраняем последовательности геномов тренировочной выборки в формате fasta

In [79]:
make_fasta_file("800", genomes_train_800)
make_fasta_file("400", genomes_train_400)

# Подготовка полногеномных выборок для поиска геномов

In [ ]:
it_0_path = "/home/parazit/ml_virus_host/v2.0/baseline/blast/it0/"
meta_df = pd.read_csv(PATH_DATA+'data_table.tsv', sep='\t', index_col = 0)
test_ids = pickle.load(open(PATH_SAMPLE_IDS+"uncrossing_genus_sample_it0.pkl", "rb"))

with open(PATH_DATA+"data.fasta", "r") as fasta_file, open(it_0_path+"test_sequences_it0.fasta", "w") as test_file, \
open(it_0_path+"train_sequences_it0.fasta", "w") as train_file:
    for seq in SeqIO.parse(fasta_file, "fasta"):
        if seq.id in test_ids:
            SeqIO.write(seq, test_file, "fasta")
        else:
            SeqIO.write(seq, train_file, "fasta")

# Подготовка выборок фрагментв для поиска фрагментов

In [116]:
from Bio.Seq import Seq

In [117]:
def fragments_fasta(fr_len, test_or_train):
    meta_df_ranged = pd.read_csv(PATH_DATA+"data_table_"+fr_len+".tsv", sep=",", index_col = 0)
    train_ids, test_ids = pickle.load(open(PATH_SAMPLE_IDS_fr+"train_val_test_"+fr_len+"_it0.pkl", "rb"))[0], \
                            pickle.load(open(PATH_SAMPLE_IDS_fr+"train_val_test_"+fr_len+"_it0.pkl", "rb"))[2]

    path_out = "/home/parazit/ml_virus_host/v2.0/baseline/blast/ranged_genus/"
    if fr_len == "800":
        path_out += "undersampled_800_7-20_f/fasta/"   
    if fr_len == "400":
        path_out += "undersampled_400_7-20_f/fasta/"
    
    if test_or_train == "test":
        records = []
        for rec, ids, descr in zip(meta_df_ranged.loc[test_ids].seq.values, meta_df_ranged.loc[test_ids].index, meta_df_ranged.loc[test_ids].host):

            records.append(SeqIO.SeqRecord(Seq(rec), ids, description=descr))

        SeqIO.write(records, path_out+"test_sequences_it0_fr.fasta", "fasta")
    
    if test_or_train == "train":
        records = []
        for rec, ids, descr in zip(meta_df_ranged.loc[train_ids].seq.values, meta_df_ranged.loc[train_ids].index, meta_df_ranged.loc[train_ids].host):
            records.append(SeqIO.SeqRecord(Seq(rec), ids, description=descr))

        SeqIO.write(records, path_out+"train_sequences_it0_fr.fasta", "fasta")
    return

In [119]:
fragments_fasta("800", "test")
fragments_fasta("800", "train")
fragments_fasta("400", "test")
fragments_fasta("400", "train")

# Анализ результатов blastn

Запуск бласт производился на сервере, выдачей бласт являлись файлы "blastn_output"

In [120]:
meta_df=pd.read_csv(PATH_DATA+'data_table.tsv', sep='\t', index_col = 'refseq id')
meta_df.head(5)

,virus tax id,virus name,virus family,host,host name,virus genus
refseq id,,,,,,
NC_001846,11138,Murine hepatitis virus,Coronaviridae,Mammalia,Mus musculus,Betacoronavirus
NC_026798,1587515,Black grass varicosavirus-like virus,Rhabdoviridae,Viridiplantae,Alopecurus myosuroides,Varicosavirus
NC_055213,2010276,Culex phasma-like virus,Phasmaviridae,Insecta,Culex quinquefasciatus,Orthophasmavirus
NC_001411,12285,Black beetle virus,Nodaviridae,Insecta,Heteronychus arator,Alphanodavirus
NC_016517,1127767,Espirito Santo virus,Birnaviridae,Insecta,Aedes albopictus,unclassified Birnaviridaevirus


In [3]:
tmp = pickle.load(open(PATH_SAMPLE_IDS+"ranged_genus/train_val_test_ids.pkl", "rb"))
train_ids, val_ids, test_ids = tmp[0], tmp[1], tmp[2]

In [4]:
meta_df = pickle.load(open(PATH_DATA+'ranged_genus/meta_df_undersampled.pkl', "rb"))
meta_df.rename(columns={"family": "virus family"}, inplace=True)
meta_df.head(5)

,host,seq,virus family,index1,index2,virus genus
gbac,,,,,,
EF065510_10,Mammalia,ctaatctagataagttgaagttcaaagaggtctgcaaaactcccac...,Coronaviridae,EF065510,EF065510_10,Betacoronavirus
AF201929_31,Mammalia,aacatttcaaaattgtaacttcgacctgagcagtctattaagattt...,Coronaviridae,AF201929,AF201929_31,Betacoronavirus
NC_014470_12,Mammalia,agttacatttagcacttttgaggaggctgctctgtgtacctttctt...,Coronaviridae,NC_014470,NC_014470_12,Betacoronavirus
EF065514_24,Mammalia,atttgagctgtgggcgaagcgatctgtaaatgtggtcccagaagtc...,Coronaviridae,EF065514,EF065514_24,Betacoronavirus
EF065510_30,Mammalia,aatcgccactctaaactttactagtcctttaacactcgcaccaata...,Coronaviridae,EF065510,EF065510_30,Betacoronavirus


In [66]:
def creat_df(finding, meta_df, func_type, l = False):
    
    if func_type:
        func = finding.pident*finding.qcovs
    else:
        func = finding.pident
    
    tmp = pd.DataFrame()
    tmp['findings'] = meta_df.loc[finding.sseqid.values].index
    tmp['host'] = meta_df.loc[finding.sseqid.values].host.values
    
    if l:
        tmp['weight'] = round(func[0], 6)
    else:
        tmp['weight'] = round(func, 6).values
    
    tmp.index = finding.index
    
    return(tmp)

Функиция для определения качества классификации методом blast

In [122]:
def blastn_analysis(iteration, weight_type, data_type = "full"):
    
    if data_type == "full":
        blastn_out = pd.read_csv(iteration+"/blastn_output_full_"+iteration+".csv", sep="\t", index_col = 'qseqid')
        future_sample_id_uncrossing = pickle.load(open(PATH_SAMPLE_IDS + 'uncrossing_genus_sample_'+iteration+'.pkl', 'rb'))
        meta_df=pd.read_csv(PATH_DATA+'data_table.tsv', sep='\t', index_col = 'refseq id')
        meta_df_X=meta_df
        save_name = iteration+"/weighted_hosts_"+iteration+".csv"
        
        print("Исходное число последовательностей: "+str(len(future_sample_id_uncrossing)))
        print("Число последовательностей с находками: "+str(len(blastn_out.index.unique()))+" ("+str(round(len(blastn_out.index.unique())/len(future_sample_id_uncrossing), 3)*100)+"%)")
    
    if data_type == "ranged":
        #blastn_out = pd.read_csv("ranged_genus/undersampled_800_7-20_f/blastn_output_800_7-20_it0.csv", sep="\t", index_col = 'qseqid')
        #future_sample_id_uncrossing = pickle.load(open(PATH_SAMPLE_IDS+"ranged_genus/undersampling_7-20_fragments/train_val_test_800_it0.pkl", "rb"))[2]
        #meta_df = pd.read_csv(PATH_DATA+"data_table_800.tsv", sep=",", index_col=0)
        #save_name = "ranged_genus/undersampled_800_7-20_f/weighted_hosts_ranged_800.csv"
        
        blastn_out = pd.read_csv("ranged_genus/undersampled_400_7-20_f/blastn_output_400_7-20_it0.csv", sep="\t", index_col = 'qseqid')
        future_sample_id_uncrossing = pickle.load(open(PATH_SAMPLE_IDS+"ranged_genus/undersampling_7-20_fragments/train_val_test_400_it0.pkl", "rb"))[2]
        meta_df = pd.read_csv(PATH_DATA+"data_table_400.tsv", sep=",", index_col=0)
        meta_df.rename(columns={"family": "virus family"}, inplace=True)
        #meta_df_X=pd.read_csv(PATH_DATA+'data_table.tsv', sep='\t', index_col = 'refseq id')
        
        
        save_name = "ranged_genus/undersampled_400_7-20_f/weighted_hosts_ranged_400.csv"
        
        print("Исходное число последовательностей: "+str(len(future_sample_id_uncrossing)))
        print("Число последовательностей с находками: "+str(len(blastn_out.index.unique()))+" ("+str(round(len(blastn_out.index.unique())/len(future_sample_id_uncrossing), 3)*100)+"%)")
    
        
    concat_me, no_findings=[], []

    for seq_id in future_sample_id_uncrossing:

        try:
            finding = blastn_out.loc[seq_id]
            tmp = creat_df(finding, meta_df, weight_type) #meta_df_X
 
        except KeyError:
            no_findings.append(seq_id)
            tmp = None
        except:
            finding = pd.DataFrame(blastn_out.loc[seq_id]).T
            tmp = creat_df(finding, meta_df, weight_type, True) #meta_df_X
        
        #print(type(tmp))
        
        if not str(type(tmp)) == "<class 'NoneType'>":
            
            
            concat_me.append(tmp)
    
    
    out = pd.concat(concat_me).reset_index().drop_duplicates().rename(columns={"index": "query_ac"}).set_index('query_ac')
    out["w"] = out.weight/out.groupby('query_ac').weight.transform('sum')
    
    k = out.groupby(['query_ac', 'host']).w.sum()
    
    a = pd.DataFrame(pd.DataFrame({"col1":list(k.index)}).col1.tolist())
    a['w']=k.values
    a = a.rename(columns={0:'AC', 1:"Host"})

    for h in ["Mammalia", "Viridiplantae", "Insecta"]:
        a[h] = list(a.Host==h)
        a[h] = a[h].map({True: 1, False: 0})

    for h in ["Mammalia", "Viridiplantae", "Insecta"]:
        a[h] = a[a.Host==h].w
    a = a.fillna(0).drop(columns=["Host", "w"])

    a.set_index("AC")
    
    a = a.groupby('AC').sum(numeric_only=True)
    a['pred host'] = a.idxmax(axis=1)
    a['virus family'] = meta_df.loc[a.index]["virus family"]
    a["real host"] = meta_df.loc[a.index]["host"]

    a = pd.concat([a, pd.DataFrame({"Mammalia":[0.000000]*len(no_findings), "Viridiplantae":[0.000000]*len(no_findings), "Insecta":[0.000000]*len(no_findings), "pred host":["unclassified"]*len(no_findings), "virus family":meta_df.loc[no_findings]["virus family"], "real host":meta_df.loc[no_findings]["host"]})])
    
    a["weight_sum"] = a.sum(axis=1, numeric_only=True).astype(int)
    a.loc[a[a.weight_sum == 0].index]["pred host"] = ["unweighted"]*len(a[a.weight_sum == 0]["pred host"])
    
    tmp = a.loc[a.weight_sum == 0]["pred host"].map({"unclassified": "unclassified", "Mammalia": "unweighted", "Viridiplantae": "unweighted", "Insecta": "unweighted",})
    a["pred host"] = pd.concat([tmp, a.loc[a.weight_sum == 1]["pred host"]])
    
    if weight_type:
        weight_type_txt = "identity * qcovs"
    if not weight_type:
        weight_type_txt = "identity"
    
    print("F1-мера = " + str(round(f1_score(y_true = a["real host"],  y_pred = a["pred host"], average="weighted"), 3)), "Вес: " + weight_type_txt)
    
    a["result"] = list(a['pred host'] == a["real host"])
    
    print("Число последовательностей с информативными находками: " + str(len(a.loc[a.weight_sum == 1])))
    print("Число последовательностей с неинформативными находками: " + str(len(a.loc[a["pred host"] == "unweighted"])))
    
    print("\nExtra info")
    print("Неверно предсказанные классы хозяев, ", dict(Counter(a[a.result==False]["real host"])))
    print("Неверно предсказанные классы хозяев в семействах вирусов, ", dict(Counter(a[a.result==False]["virus family"])))

    print("Семейства вирусов, для которых бласт не нашёл схожие геномы, ", dict(Counter(meta_df.loc[no_findings]["virus family"])))
    print("Классы хозяев в семействах вирусов, для которых бласт не нашёл схожие геномы, ", dict(Counter(meta_df.loc[no_findings]["host"])))
    
    print(classification_report(a["real host"], a["pred host"], zero_division=True))
    
    #a.to_csv(save_name, sep='\t')
    
    return a

Код для определения качества классификации методом blast для случая, когда обучение проводится на геномах, а тестирование - на фрагментах разных длин

In [97]:
fragment_len = "400" # длина используемого фрагмента

if fragment_len == "400":
    list_of_fragmnets = fr400_test
if fragment_len == "800":
    list_of_fragmnets = fr800_test
    
blastn_out = pd.read_csv(PATH_GENOMES2FRAGMENTS+"blastn_output_"+fragment_len+".csv", sep="\t", index_col = 'qseqid')
weight_type = 1
meta_df_ranged = pd.read_csv(PATH_DATA+"data_table_"+fragment_len+".tsv", sep=",", index_col = 0)

concat_me, no_findings=[], []

for seq_id in list_of_fragmnets:

    try:
        finding = blastn_out.loc[seq_id]
        tmp = creat_df(finding, meta_df, weight_type) #meta_df_X

    except KeyError:
        no_findings.append(seq_id)
        tmp = None
    except:
        finding = pd.DataFrame(blastn_out.loc[seq_id]).T
        tmp = creat_df(finding, meta_df, weight_type, True) #meta_df_X

    #print(type(tmp))

    if not str(type(tmp)) == "<class 'NoneType'>":


        concat_me.append(tmp)
        
        
out = pd.concat(concat_me).reset_index().drop_duplicates().rename(columns={"index": "query_ac"}).set_index('query_ac')
out["w"] = out.weight/out.groupby('query_ac').weight.transform('sum')

k = out.groupby(['query_ac', 'host']).w.sum()

a = pd.DataFrame(pd.DataFrame({"col1":list(k.index)}).col1.tolist())
a['w']=k.values
a = a.rename(columns={0:'AC', 1:"Host"})

for h in ["Mammalia", "Viridiplantae", "Insecta"]:
    a[h] = list(a.Host==h)
    a[h] = a[h].map({True: 1, False: 0})

for h in ["Mammalia", "Viridiplantae", "Insecta"]:
    a[h] = a[a.Host==h].w
a = a.fillna(0).drop(columns=["Host", "w"])

a.set_index("AC")

a = a.groupby('AC').sum(numeric_only=True)
a['pred host'] = a.idxmax(axis=1)
a['virus family'] = meta_df_ranged.loc[a.index]["family"]
a["real host"] = meta_df_ranged.loc[a.index]["host"]

a = pd.concat([a, pd.DataFrame({"Mammalia":[0.000000]*len(no_findings), "Viridiplantae":[0.000000]*len(no_findings), "Insecta":[0.000000]*len(no_findings), "pred host":["unclassified"]*len(no_findings), "virus family":meta_df_ranged.loc[no_findings]["family"], "real host":meta_df_ranged.loc[no_findings]["host"]})])

a["weight_sum"] = a.sum(axis=1, numeric_only=True).astype(int)
a.loc[a[a.weight_sum == 0].index]["pred host"] = ["unweighted"]*len(a[a.weight_sum == 0]["pred host"])

tmp = a.loc[a.weight_sum == 0]["pred host"].map({"unclassified": "unclassified", "Mammalia": "unweighted", "Viridiplantae": "unweighted", "Insecta": "unweighted",})
a["pred host"] = pd.concat([tmp, a.loc[a.weight_sum == 1]["pred host"]])

if weight_type:
    weight_type_txt = "identity * qcovs"
if not weight_type:
    weight_type_txt = "identity"

print("F1-мера = " + str(round(f1_score(y_true = a["real host"],  y_pred = a["pred host"], average="weighted"), 3)), "Вес: " + weight_type_txt)

a["result"] = list(a['pred host'] == a["real host"])
print("Число последовательностей с информативными находками: " + str(len(a.loc[a.weight_sum == 1])))
print("Число последовательностей с неинформативными находками: " + str(len(a.loc[a["pred host"] == "unweighted"])))

print("\nExtra info")
print("Неверно предсказанные классы хозяев, ", dict(Counter(a[a.result==False]["real host"])))
print("Неверно предсказанные классы хозяев в семействах вирусов, ", dict(Counter(a[a.result==False]["virus family"])))

print("Семейства вирусов, для которых бласт не нашёл схожие геномы, ", dict(Counter(meta_df_ranged.loc[no_findings]["family"])))
print("Классы хозяев в семействах вирусов, для которых бласт не нашёл схожие геномы, ", dict(Counter(meta_df_ranged.loc[no_findings]["host"])))

print(classification_report(a["real host"], a["pred host"], zero_division=True))
    

F1-мера = 0.348 Вес: identity * qcovs
Число последовательностей с информативными находками: 91
Число последовательностей с неинформативными находками: 5

Extra info
Неверно предсказанные классы хозяев,  {'Mammalia': 76, 'Viridiplantae': 103, 'Insecta': 126}
Неверно предсказанные классы хозяев в семействах вирусов,  {'Picornaviridae': 24, 'Secoviridae': 52, 'Rhabdoviridae': 25, 'Solinviviridae': 7, 'Caliciviridae': 25, 'Bromoviridae': 19, 'Paramyxoviridae': 6, 'Mesoniviridae': 7, 'Phenuiviridae': 12, 'Chrysoviridae': 7, 'Spinareoviridae': 7, 'Astroviridae': 10, 'Hepeviridae': 3, 'Permutotetraviridae': 7, 'Dicistroviridae': 6, 'Peribunyaviridae': 7, 'Phasmaviridae': 13, 'Potyviridae': 32, 'Parvoviridae': 7, 'Mitoviridae': 7, 'Togaviridae': 7, 'Xinmoviridae': 6, 'Flaviviridae': 2, 'Alphatetraviridae': 7}
Семейства вирусов, для которых бласт не нашёл схожие геномы,  {'Picornaviridae': 21, 'Caliciviridae': 25, 'Bromoviridae': 19, 'Paramyxoviridae': 6, 'Mesoniviridae': 7, 'Phenuiviridae': 12

In [127]:
blastn_analysis("it2", 0, "full")

Исходное число последовательностей: 287
Число последовательностей с находками: 255 (88.9%)
F1-мера = 0.709 Вес: identity
Число последовательностей с информативными находками: 218
Число последовательностей с неинформативными находками: 37

Extra info
Неверно предсказанные классы хозяев,  {'Mammalia': 33, 'Viridiplantae': 27, 'Insecta': 47}
Неверно предсказанные классы хозяев в семействах вирусов,  {'Picornaviridae': 8, 'Coronaviridae': 2, 'Bromoviridae': 1, 'Rhabdoviridae': 48, 'Secoviridae': 11, 'Dicistroviridae': 2, 'Parvoviridae': 1, 'Potyviridae': 1, 'Paramyxoviridae': 1, 'Iflaviridae': 7, 'Xinmoviridae': 1, 'Alphatetraviridae': 3, 'Spinareoviridae': 10, 'Chuviridae': 1, 'Solinviviridae': 1, 'Sedoreoviridae': 1, 'Sinhaliviridae': 7, 'Reoviridae': 1}
Семейства вирусов, для которых бласт не нашёл схожие геномы,  {'Alphatetraviridae': 3, 'Secoviridae': 4, 'Spinareoviridae': 10, 'Picornaviridae': 2, 'Chuviridae': 1, 'Rhabdoviridae': 1, 'Solinviviridae': 1, 'Sedoreoviridae': 1, 'Sinhaliv

,Mammalia,Viridiplantae,Insecta,pred host,virus family,real host,weight_sum,result
AB252582,1.0,0.0,0.0,Mammalia,Picornaviridae,Mammalia,1,True
AF274010,1.0,0.0,0.0,Mammalia,Picornaviridae,Mammalia,1,True
AF327920,1.0,0.0,0.0,Mammalia,Picornaviridae,Mammalia,1,True
AF327922,1.0,0.0,0.0,Mammalia,Picornaviridae,Mammalia,1,True
AF338106,1.0,0.0,0.0,Mammalia,Paramyxoviridae,Mammalia,1,True
...,...,...,...,...,...,...,...,...
NC_035111,0.0,0.0,0.0,unclassified,Sinhaliviridae,Insecta,0,False
NC_035112,0.0,0.0,0.0,unclassified,Sinhaliviridae,Insecta,0,False
NC_035116,0.0,0.0,0.0,unclassified,Sinhaliviridae,Insecta,0,False
NC_040498,0.0,0.0,0.0,unclassified,Reoviridae,Insecta,0,False
